# OTP vs. Load Factor Analysis: Southwest's Top 5 Routes

This notebook analyzes the relationship between **Southwest Airlines' On-Time Performance (OTP)** and **Load Factor** on the top 5 domestic routes.

**Research Question:**  
How does Southwest's OTP relate to the Load Factor of its flights on the top 5 domestic routes?

---

## Import Libraries and Setup


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import sys
sys.path.append('..')

from src import MetricsCalculator

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)

# Set plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

# %matplotlib inline

---


### Cell 1. Load Top 5 Routes Data

In [ ]:
# Load processed data
lf_top5 = pd.read_csv('../data/processed/lf_top5_routes.csv')
otp_top5 = pd.read_csv('../data/processed/otp_top5_routes.csv')
top_routes = pd.read_csv('../data/processed/top5_routes.csv')['ROUTE'].tolist()

# Convert dates
lf_top5['DATE'] = pd.to_datetime(lf_top5['DATE'])
otp_top5['DATE'] = pd.to_datetime(otp_top5['DATE'])

print(f"Top 5 Routes: {', '.join(top_routes)}")
print(f"\nLoad Factor records: {len(lf_top5):,}")
print(f"OTP records: {len(otp_top5):,}")

---


### Cell 2. Merge Load Factor and OTP Data

In [ ]:
# Use MetricsCalculator to merge data
calc = MetricsCalculator()
merged_data = calc.merge_lf_otp_by_route_month(lf_top5, otp_top5)

print(f"Merged dataset shape: {merged_data.shape}")
print(f"Date range: {merged_data['DATE'].min()} to {merged_data['DATE'].max()}")
print(f"\nRoutes in merged data: {merged_data['ROUTE'].nunique()}")
merged_data.head(10)


# Check data availability by route
route_counts = merged_data.groupby('ROUTE').size().sort_values(ascending=False)
print("Number of months with data by route:")
print(route_counts)

---


### Cell 3. Calculate Summary Statistics by Route

In [ ]:
# Calculate summary statistics for each route
summary_stats = []

for route in top_routes:
    stats = calc.route_summary_stats(merged_data, route)
    if stats:
        summary_stats.append(stats)

summary_df = pd.DataFrame(summary_stats)
summary_df = summary_df.sort_values('total_flights', ascending=False)

print("\nSummary Statistics by Route:")
print("="*100)
summary_df


# Display correlations more clearly
print("\nLoad Factor vs OTP Correlations:")
print("="*80)
correlation_summary = summary_df[[
    'route', 'corr_lf_dep_ontime', 'corr_lf_dep_pvalue', 
    'corr_lf_arr_ontime', 'corr_lf_arr_pvalue'
]].copy()

# Add significance markers
def add_significance(row):
    dep_sig = '***' if row['corr_lf_dep_pvalue'] < 0.001 else \
              '**' if row['corr_lf_dep_pvalue'] < 0.01 else \
              '*' if row['corr_lf_dep_pvalue'] < 0.05 else 'ns'
    arr_sig = '***' if row['corr_lf_arr_pvalue'] < 0.001 else \
              '**' if row['corr_lf_arr_pvalue'] < 0.01 else \
              '*' if row['corr_lf_arr_pvalue'] < 0.05 else 'ns'
    return pd.Series([dep_sig, arr_sig])

correlation_summary[['dep_sig', 'arr_sig']] = correlation_summary.apply(add_significance, axis=1)
print(correlation_summary)
print("\nSignificance: *** p<0.001, ** p<0.01, * p<0.05, ns = not significant")


---


### Cell 4. Visualize Overall Relationship

In [ ]:
# Scatter plot: Load Factor vs Departure On-Time Performance (All Routes)
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Departure OTP
for route in top_routes:
    route_data = merged_data[merged_data['ROUTE'] == route]
    axes[0].scatter(route_data['LOAD_FACTOR'], route_data['DEP_ONTIME_PCT'], 
                    label=route, alpha=0.6, s=50)

axes[0].set_xlabel('Load Factor (%)', fontsize=12)
axes[0].set_ylabel('Departure On-Time Performance (%)', fontsize=12)
axes[0].set_title('Load Factor vs Departure OTP', fontsize=13, fontweight='bold')
axes[0].legend(loc='best', fontsize=9)
axes[0].grid(True, alpha=0.3)

# Arrival OTP
for route in top_routes:
    route_data = merged_data[merged_data['ROUTE'] == route]
    axes[1].scatter(route_data['LOAD_FACTOR'], route_data['ARR_ONTIME_PCT'], 
                    label=route, alpha=0.6, s=50)

axes[1].set_xlabel('Load Factor (%)', fontsize=12)
axes[1].set_ylabel('Arrival On-Time Performance (%)', fontsize=12)
axes[1].set_title('Load Factor vs Arrival OTP', fontsize=13, fontweight='bold')
axes[1].legend(loc='best', fontsize=9)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---


### Cell 5. Individual Route Analysis

In [ ]:
# Create detailed plots for each route
for route in top_routes:
    route_data = merged_data[merged_data['ROUTE'] == route]
    
    if len(route_data) == 0:
        continue
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle(f'Route: {route}', fontsize=16, fontweight='bold', y=1.00)
    
    # Plot 1: Time series - Load Factor
    ax1 = axes[0, 0]
    ax1.plot(route_data['DATE'], route_data['LOAD_FACTOR'], 
             marker='o', linewidth=2, markersize=5, color='#FF6B35')
    ax1.set_xlabel('Date', fontsize=11)
    ax1.set_ylabel('Load Factor (%)', fontsize=11)
    ax1.set_title('Load Factor Over Time', fontsize=12, fontweight='bold')
    ax1.grid(True, alpha=0.3)
    ax1.axhline(y=route_data['LOAD_FACTOR'].mean(), color='red', 
                linestyle='--', alpha=0.5, label=f"Mean: {route_data['LOAD_FACTOR'].mean():.1f}%")
    ax1.legend()
    
    # Plot 2: Time series - OTP
    ax2 = axes[0, 1]
    ax2.plot(route_data['DATE'], route_data['DEP_ONTIME_PCT'], 
             marker='o', linewidth=2, markersize=5, label='Departure', color='#004E89')
    ax2.plot(route_data['DATE'], route_data['ARR_ONTIME_PCT'], 
             marker='s', linewidth=2, markersize=5, label='Arrival', color='#F77F00')
    ax2.set_xlabel('Date', fontsize=11)
    ax2.set_ylabel('On-Time Performance (%)', fontsize=11)
    ax2.set_title('On-Time Performance Over Time', fontsize=12, fontweight='bold')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # Plot 3: Scatter - Load Factor vs Departure OTP
    ax3 = axes[1, 0]
    ax3.scatter(route_data['LOAD_FACTOR'], route_data['DEP_ONTIME_PCT'], 
                alpha=0.6, s=80, color='#004E89')
    
    # Add regression line
    z = np.polyfit(route_data['LOAD_FACTOR'], route_data['DEP_ONTIME_PCT'], 1)
    p = np.poly1d(z)
    ax3.plot(route_data['LOAD_FACTOR'], p(route_data['LOAD_FACTOR']), 
             "r--", alpha=0.8, linewidth=2)
    
    corr = calc.calculate_correlation(route_data, 'LOAD_FACTOR', 'DEP_ONTIME_PCT')
    ax3.set_xlabel('Load Factor (%)', fontsize=11)
    ax3.set_ylabel('Departure On-Time (%)', fontsize=11)
    ax3.set_title(f"Load Factor vs Departure OTP\n" +
                  f"Correlation: {corr['correlation']:.3f} (p={corr['p_value']:.4f})", 
                  fontsize=12, fontweight='bold')
    ax3.grid(True, alpha=0.3)
    
    # Plot 4: Scatter - Load Factor vs Arrival OTP
    ax4 = axes[1, 1]
    ax4.scatter(route_data['LOAD_FACTOR'], route_data['ARR_ONTIME_PCT'], 
                alpha=0.6, s=80, color='#F77F00')
    
    # Add regression line
    z = np.polyfit(route_data['LOAD_FACTOR'], route_data['ARR_ONTIME_PCT'], 1)
    p = np.poly1d(z)
    ax4.plot(route_data['LOAD_FACTOR'], p(route_data['LOAD_FACTOR']), 
             "r--", alpha=0.8, linewidth=2)
    
    corr = calc.calculate_correlation(route_data, 'LOAD_FACTOR', 'ARR_ONTIME_PCT')
    ax4.set_xlabel('Load Factor (%)', fontsize=11)
    ax4.set_ylabel('Arrival On-Time (%)', fontsize=11)
    ax4.set_title(f"Load Factor vs Arrival OTP\n" +
                  f"Correlation: {corr['correlation']:.3f} (p={corr['p_value']:.4f})", 
                  fontsize=12, fontweight='bold')
    ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n{'='*80}")
    print(f"Route {route} - Key Metrics:")
    print(f"{'='*80}")
    print(f"Average Load Factor: {route_data['LOAD_FACTOR'].mean():.2f}%")
    print(f"Average Departure OTP: {route_data['DEP_ONTIME_PCT'].mean():.2f}%")
    print(f"Average Arrival OTP: {route_data['ARR_ONTIME_PCT'].mean():.2f}%")
    print(f"Total Flights Analyzed: {route_data['TOTAL_FLIGHTS'].sum():,.0f}")
    print(f"\n")

---


### Cell 6. Correlation Heatmap

In [ ]:
# Create correlation matrix for key variables
corr_vars = ['LOAD_FACTOR', 'DEP_ONTIME_PCT', 'ARR_ONTIME_PCT', 
             'CANCELLATION_PCT', 'TOTAL_FLIGHTS']
corr_matrix = merged_data[corr_vars].corr()

# Plot heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, fmt='.3f', cmap='coolwarm', 
            center=0, square=True, linewidths=1,
            cbar_kws={"shrink": 0.8})
plt.title('Correlation Matrix: Load Factor and OTP Metrics (All Top 5 Routes)', 
          fontsize=14, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

---


### Cell 7. Load Factor Bins Analysis

In [ ]:
# Create load factor bins
merged_data['LF_BIN'] = pd.cut(merged_data['LOAD_FACTOR'], 
                                bins=[0, 70, 75, 80, 85, 100],
                                labels=['<70%', '70-75%', '75-80%', '80-85%', '85%+'])

# Calculate average OTP by load factor bin
bin_analysis = merged_data.groupby('LF_BIN').agg({
    'DEP_ONTIME_PCT': ['mean', 'std'],
    'ARR_ONTIME_PCT': ['mean', 'std'],
    'CANCELLATION_PCT': 'mean',
    'TOTAL_FLIGHTS': 'sum'
}).round(2)

print("OTP by Load Factor Bin:")
print(bin_analysis)


# Visualize OTP by load factor bins
bin_summary = merged_data.groupby('LF_BIN').agg({
    'DEP_ONTIME_PCT': 'mean',
    'ARR_ONTIME_PCT': 'mean'
}).reset_index()

fig, ax = plt.subplots(figsize=(12, 6))
x = np.arange(len(bin_summary))
width = 0.35

ax.bar(x - width/2, bin_summary['DEP_ONTIME_PCT'], width, 
       label='Departure OTP', color='#004E89')
ax.bar(x + width/2, bin_summary['ARR_ONTIME_PCT'], width, 
       label='Arrival OTP', color='#F77F00')

ax.set_xlabel('Load Factor Bin', fontsize=12)
ax.set_ylabel('On-Time Performance (%)', fontsize=12)
ax.set_title('Average OTP by Load Factor Bin', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(bin_summary['LF_BIN'])
ax.legend()
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

---


### Cell 8. Statistical Testing

In [ ]:
# Perform statistical tests for overall relationship
print("Overall Statistical Analysis (All Top 5 Routes Combined)")
print("="*80)

# Pearson correlation
pearson_dep = calc.calculate_correlation(merged_data, 'LOAD_FACTOR', 'DEP_ONTIME_PCT', method='pearson')
pearson_arr = calc.calculate_correlation(merged_data, 'LOAD_FACTOR', 'ARR_ONTIME_PCT', method='pearson')

print("\nLoad Factor vs Departure OTP:")
print(f"  Pearson r = {pearson_dep['correlation']:.4f}, p = {pearson_dep['p_value']:.6f}, n = {pearson_dep['n']}")

print("\nLoad Factor vs Arrival OTP:")
print(f"  Pearson r = {pearson_arr['correlation']:.4f}, p = {pearson_arr['p_value']:.6f}, n = {pearson_arr['n']}")

# Spearman correlation (non-parametric)
spearman_dep = calc.calculate_correlation(merged_data, 'LOAD_FACTOR', 'DEP_ONTIME_PCT', method='spearman')
spearman_arr = calc.calculate_correlation(merged_data, 'LOAD_FACTOR', 'ARR_ONTIME_PCT', method='spearman')

print("\nSpearman Rank Correlation (Load Factor vs Departure OTP):")
print(f"  Spearman ρ = {spearman_dep['correlation']:.4f}, p = {spearman_dep['p_value']:.6f}")

print("\nSpearman Rank Correlation (Load Factor vs Arrival OTP):")
print(f"  Spearman ρ = {spearman_arr['correlation']:.4f}, p = {spearman_arr['p_value']:.6f}")

---


### Cell 9. Save Results

In [ ]:
# Save merged data and summary statistics
import os
merged_path = '../data/processed/merged_lf_otp_top5.csv'
summary_path = '../data/processed/route_summary_statistics.csv'

if os.path.exists(merged_path) and os.path.exists(summary_path):
    print("Merged data and summary statistics already exist in data/processed/. Skipping save.")
else:
    merged_data.to_csv(merged_path, index=False)
    summary_df.to_csv(summary_path, index=False)
    print("Analysis results saved to data/processed/")

---


## Key Findings

This analysis investigates the relationship between **Load Factor** and **On-Time Performance (OTP)** for Southwest Airlines' top 5 domestic routes from **2020 to 2025**.

### Statistical Summary:
- The **correlation analysis** highlights the strength and direction of the relationship between **Load Factor** and **OTP**.
- **Route-specific patterns** exhibit varying relationships across different markets.
- The **Load Factor bins analysis** demonstrates how OTP fluctuates at different levels of capacity utilization.

**Note:** Detailed findings and business implications will be included in the final report.


---


### Please see the Final Report in " `reports/final_report_completed.md` "

#### Note: Running the Python file "generate_report.py" will automatically generate a copy of the report. However, the Final Report *is* alreayd provided.